In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import GridSearchCV, KFold
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler

In [ ]:
df=pd.read_csv("train.csv")
dft=pd.read_csv("test.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12841 entries, 0 to 12840
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   job title                   12841 non-null  object
 1   office location             12582 non-null  object
 2   department                  4581 non-null   object
 3   salary offered for the job  2089 non-null   object
 4   company information         10436 non-null  object
 5   job description             12841 non-null  object
 6   job requirements            10900 non-null  object
 7   benefits                    7596 non-null   object
 8   telecommuting               12841 non-null  int64 
 9   company logo exist?         12841 non-null  int64 
 10  has_questions               12841 non-null  int64 
 11  employment_type             10313 non-null  object
 12  experience required         7755 non-null   object
 13  education required          6988 non-null   ob

In [ ]:
dft.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3198 entries, 0 to 3197
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   job title                   3198 non-null   object
 1   office location             3153 non-null   object
 2   department                  1115 non-null   object
 3   salary offered for the job  505 non-null    object
 4   company information         2621 non-null   object
 5   job description             3197 non-null   object
 6   job requirements            2729 non-null   object
 7   benefits                    1976 non-null   object
 8   telecommuting               3198 non-null   int64 
 9   company logo exist?         3198 non-null   int64 
 10  has_questions               3198 non-null   int64 
 11  employment_type             2627 non-null   object
 12  experience required         1973 non-null   object
 13  education required          1792 non-null   obje

In [ ]:
print (df.isnull().sum())
print (df.isnull().values.any())
print (df.isnull().values.sum())

job title                         0
office location                 259
department                     8260
salary offered for the job    10752
company information            2405
job description                   0
job requirements               1941
benefits                       5245
telecommuting                     0
company logo exist?               0
has_questions                     0
employment_type                2528
experience required            5086
education required             5853
industry                       3546
function                       4659
fake?                             0
dtype: int64
True
50534


In [ ]:
df["office location"] = df["office location"].fillna(" ")
df["department"] = df["department"].fillna(" ")
df["job title"] = df["job title"].fillna(" ")
df["company information"] = df["company information"].fillna(" ")
df["job requirements"] = df["job requirements"].fillna(" ")
df["benefits"] = df["benefits"].fillna(" ")

In [ ]:
dft["office location"] = dft["office location"].fillna(" ")
dft["department"] = dft["department"].fillna(" ")
dft["job title"] = dft["job title"].fillna(" ")
dft["company information"] = dft["company information"].fillna(" ")
dft["job requirements"] = dft["job requirements"].fillna(" ")
dft["benefits"] = dft["benefits"].fillna(" ")

In [ ]:
df["job2"] = df["job title"].str.cat(df["office location"], sep = " ")
df["job3"] = df["department"].str.cat(df["company information"], sep = " ")
df["job4"] = df["job description"].str.cat(df["job requirements"], sep = " ")
df["job5"] = df["job4"].str.cat(df["benefits"], sep = " ")

In [ ]:
dft["job2"] = dft["job title"].str.cat(dft["office location"], sep = " ")
dft["job3"] = dft["department"].str.cat(dft["company information"], sep = " ")
dft["job4"] = dft["job description"].str.cat(dft["job requirements"], sep = " ")
dft["job5"] = dft["job4"].str.cat(dft["benefits"], sep = " ")

In [ ]:
df = df.drop("office location", axis=1)
df = df.drop("department", axis=1)
df = df.drop("job title", axis=1)
df = df.drop("company information", axis=1)
df = df.drop("job description", axis=1)
df = df.drop("job requirements", axis=1)
df = df.drop("benefits", axis=1)

In [ ]:
dft = dft.drop("office location", axis=1)
dft = dft.drop("department", axis=1)
dft = dft.drop("job title", axis=1)
dft = dft.drop("company information", axis=1)
dft = dft.drop("job description", axis=1)
dft = dft.drop("job requirements", axis=1)
dft = dft.drop("benefits", axis=1)

In [ ]:
df.head()

,salary offered for the job,telecommuting,company logo exist?,has_questions,employment_type,experience required,education required,industry,function,fake?,job2,job3,job4,job5
0,NaN,0,1,1,Full-time,Not Applicable,Unspecified,Apparel & Fashion,Art/Creative,no,"Videographer / Editor / Photographer GB, LND, ...",Dreamers &amp; Creators is a creative agency...,The Videographer / Editor / Photographer will ...,The Videographer / Editor / Photographer will ...
1,NaN,0,1,1,NaN,NaN,NaN,NaN,NaN,no,"Supply Chain Manager GR, I, Kifisia",Wholesale Η πρώτη στεγασμένη αγορά παραγωγών μ...,Buyer will be in charge of expanding our vendo...,Buyer will be in charge of expanding our vendo...
2,NaN,0,1,1,NaN,NaN,NaN,NaN,NaN,no,"Digital Project Manager US, CA, El Segundo",Product Development ConsumerTrack is a technol...,We are looking for a Digital Project Manager t...,We are looking for a Digital Project Manager t...
3,NaN,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Internet,Marketing,no,"Global Communications Manager GB, LND, London",Marketing Founded in 2010 by a team from Googl...,"Qubit: Cutting Edge Big Data EngineeringQubit,...","Qubit: Cutting Edge Big Data EngineeringQubit,..."
4,NaN,0,1,0,Full-time,NaN,NaN,Information Technology and Services,NaN,no,"S3I Candidates LT, ,",All,Adform is a premier partner for media agencies...,Adform is a premier partner for media agencies...


In [ ]:
df = df.drop("job2", axis=1)
df = df.drop("job3", axis=1)
df = df.drop("job4", axis=1)

In [ ]:
dft = dft.drop("job2", axis=1)
dft = dft.drop("job3", axis=1)
dft = dft.drop("job4", axis=1)

In [ ]:
df.rename(columns = {'job5':'job'}, inplace = True)

In [ ]:
dft.rename(columns = {'job5':'job'}, inplace = True)

In [ ]:
df.head()

,salary offered for the job,telecommuting,company logo exist?,has_questions,employment_type,experience required,education required,industry,function,fake?,job
0,NaN,0,1,1,Full-time,Not Applicable,Unspecified,Apparel & Fashion,Art/Creative,no,The Videographer / Editor / Photographer will ...
1,NaN,0,1,1,NaN,NaN,NaN,NaN,NaN,no,Buyer will be in charge of expanding our vendo...
2,NaN,0,1,1,NaN,NaN,NaN,NaN,NaN,no,We are looking for a Digital Project Manager t...
3,NaN,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Internet,Marketing,no,"Qubit: Cutting Edge Big Data EngineeringQubit,..."
4,NaN,0,1,0,Full-time,NaN,NaN,Information Technology and Services,NaN,no,Adform is a premier partner for media agencies...


In [ ]:
df.drop("salary offered for the job", axis=1)

,telecommuting,company logo exist?,has_questions,employment_type,experience required,education required,industry,function,fake?,job
0,0,1,1,Full-time,Not Applicable,Unspecified,Apparel & Fashion,Art/Creative,no,The Videographer / Editor / Photographer will ...
1,0,1,1,NaN,NaN,NaN,NaN,NaN,no,Buyer will be in charge of expanding our vendo...
2,0,1,1,NaN,NaN,NaN,NaN,NaN,no,We are looking for a Digital Project Manager t...
3,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Internet,Marketing,no,"Qubit: Cutting Edge Big Data EngineeringQubit,..."
4,0,1,0,Full-time,NaN,NaN,Information Technology and Services,NaN,no,Adform is a premier partner for media agencies...
...,...,...,...,...,...,...,...,...,...,...
12836,0,1,0,NaN,NaN,NaN,NaN,NaN,no,PURPOSEResponsible for selling skincare produc...
12837,0,1,1,Full-time,Mid-Senior level,NaN,Internet,Project Management,no,Mindworks is looking for a Senior Digital Acco...
12838,0,1,1,Full-time,NaN,Associate Degree,NaN,Design,no,Schroeder Solutions is a full-service interior...
12839,0,0,0,Full-time,NaN,NaN,Computer Software,NaN,no,(We have more than 1500+ Job openings in our w...


In [ ]:
dft.drop("salary offered for the job", axis=1)

,telecommuting,company logo exist?,has_questions,employment_type,experience required,education required,industry,function,fake?,job
0,0,0,1,Full-time,NaN,Unspecified,Construction,Management,no,Seeking qualified candidates for a full-time s...
1,0,1,1,Full-time,Mid-Senior level,NaN,Real Estate,NaN,no,Looking for a change? Not happy where you are...
2,0,1,0,Full-time,Associate,NaN,Computer Software,Information Technology,no,About the CompanyWe are ticketscript - the Eur...
3,0,1,0,Full-time,Not Applicable,Unspecified,Information Technology and Services,Information Technology,no,#NAME? - Must have extensive knowledge of CMS ...
4,0,1,1,Full-time,NaN,NaN,NaN,NaN,no,We are looking for a Program Manager who can h...
...,...,...,...,...,...,...,...,...,...,...
3193,0,1,1,Full-time,Entry level,NaN,Financial Services,Finance,no,Who are we?Proficio Mortgage Ventures is a who...
3194,0,0,1,NaN,NaN,NaN,"Leisure, Travel & Tourism",NaN,yes,6* Ultra Luxury American Cruise Company is urg...
3195,0,1,1,Full-time,Entry level,High School or equivalent,Civic & Social Organization,Health Care Provider,no,Are you looking for full-time work with great ...
3196,0,0,0,Full-time,Mid-Senior level,Unspecified,Financial Services,Management,no,"Westview Financial Services, located in Hampto..."


In [ ]:
df["education required"] = df["education required"].fillna(" ")
df["employment_type"] = df["employment_type"].fillna(" ")
df["experience required"] = df["experience required"].fillna(" ")
df["industry"] = df["industry"].fillna(" ")
df["function"] = df["function"].fillna(" ")

In [ ]:
dft["education required"] = dft["education required"].fillna(" ")
dft["employment_type"] = dft["employment_type"].fillna(" ")
dft["experience required"] = dft["experience required"].fillna(" ")
dft["industry"] = dft["industry"].fillna(" ")
dft["function"] = dft["function"].fillna(" ")

In [ ]:
df["req1"] = df["education required"].str.cat(df["employment_type"], sep = " ")
df["requirements"] = df["req1"].str.cat(df["experience required"], sep = " ")
df["category"] = df["industry"].str.cat(df["function"], sep = " ")

In [ ]:
dft["req1"] = dft["education required"].str.cat(df["employment_type"], sep = " ")
dft["requirements"] = dft["req1"].str.cat(df["experience required"], sep = " ")
dft["category"] = dft["industry"].str.cat(df["function"], sep = " ")

In [ ]:
df["job"] = df["job"].str.cat(df["category"], sep = " ")
df["job"] = df["job"].str.cat(df["requirements"], sep = " ")

In [ ]:
dft["job"] = dft["job"].str.cat(dft["category"], sep = " ")
dft["job"] = dft["job"].str.cat(dft["requirements"], sep = " ")

In [ ]:
df=df.drop("req1", axis=1)

In [ ]:
dft=dft.drop("req1", axis=1)

In [ ]:
df.head()

,salary offered for the job,telecommuting,company logo exist?,has_questions,employment_type,experience required,education required,industry,function,fake?,job,requirements,category
0,NaN,0,1,1,Full-time,Not Applicable,Unspecified,Apparel & Fashion,Art/Creative,no,The Videographer / Editor / Photographer will ...,Unspecified Full-time Not Applicable,Apparel & Fashion Art/Creative
1,NaN,0,1,1,,,,,,no,Buyer will be in charge of expanding our vendo...,,
2,NaN,0,1,1,,,,,,no,We are looking for a Digital Project Manager t...,,
3,NaN,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Internet,Marketing,no,"Qubit: Cutting Edge Big Data EngineeringQubit,...",Bachelor's Degree Full-time Mid-Senior level,Internet Marketing
4,NaN,0,1,0,Full-time,,,Information Technology and Services,,no,Adform is a premier partner for media agencies...,Full-time,Information Technology and Services


In [ ]:
df=df.drop("employment_type", axis=1)
df=df.drop("experience required", axis=1)
df=df.drop("education required", axis=1)
df=df.drop("industry", axis=1)
df=df.drop("function", axis=1)
df=df.drop("requirements", axis=1)
df=df.drop("category", axis=1)
df=df.drop("salary offered for the job", axis=1)

In [ ]:
dft=dft.drop("employment_type", axis=1)
dft=dft.drop("experience required", axis=1)
dft=dft.drop("education required", axis=1)
dft=dft.drop("industry", axis=1)
dft=dft.drop("function", axis=1)
dft=dft.drop("requirements", axis=1)
dft=dft.drop("category", axis=1)
dft=dft.drop("salary offered for the job", axis=1)

In [ ]:
df.head()

,telecommuting,company logo exist?,has_questions,fake?,job
0,0,1,1,no,The Videographer / Editor / Photographer will ...
1,0,1,1,no,Buyer will be in charge of expanding our vendo...
2,0,1,1,no,We are looking for a Digital Project Manager t...
3,0,1,1,no,"Qubit: Cutting Edge Big Data EngineeringQubit,..."
4,0,1,0,no,Adform is a premier partner for media agencies...


In [ ]:
dft.head()

,telecommuting,company logo exist?,has_questions,fake?,job
0,0,0,1,no,Seeking qualified candidates for a full-time s...
1,0,1,1,no,Looking for a change? Not happy where you are...
2,0,1,0,no,About the CompanyWe are ticketscript - the Eur...
3,0,1,0,no,#NAME? - Must have extensive knowledge of CMS ...
4,0,1,1,no,We are looking for a Program Manager who can h...


In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
#df['job'].astype(str)
dft['job'].astype(str)

0       Seeking qualified candidates for a full-time s...
1       Looking for a change?  Not happy where you are...
2       About the CompanyWe are ticketscript - the Eur...
3       #NAME? - Must have extensive knowledge of CMS ...
4       We are looking for a Program Manager who can h...
                              ...                        
3193    Who are we?Proficio Mortgage Ventures is a who...
3194    6* Ultra Luxury American Cruise Company is urg...
3195    Are you looking for full-time work with great ...
3196    Westview Financial Services, located in Hampto...
3197    Funnelback is an exciting and growing company ...
Name: job, Length: 3198, dtype: object

In [ ]:
# Text cleaning
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    if type(text) != str:
        return ""
    text = re.sub('(@|#)\w+|[,.]|[()]|https?://\S+|(\?|!)+|\d+',' ',text)
    text = text.lower()
    text = ' '.join([word for word in text.split() if word not in stop_words])
    text = ' '.join([stemmer.stem(word) for word in text.split()])
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
    return text

df['clean_text'] = df['job'].apply(clean_text)
dft['clean_text'] = dft['job'].apply(clean_text)

In [ ]:
# def clean_text(text):
#     tokens = word_tokenize(text.lower())
#     stop_words = set(stopwords.words('english'))
#     filtered_tokens = [token for token in tokens if token not in stop_words]
#     lemmatizer = WordNetLemmatizer()
#     lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
#     cleaned_tokens = []
#     for token in lemmatized_tokens:
#         cleaned_token = ''.join(char for char in token if char not in string.punctuation)
#         if cleaned_token:
#             cleaned_tokens.append(cleaned_token)
#     cleaned_text = ' '.join(cleaned_tokens)
#     return cleaned_text

# df['job'] = df['job'].apply(clean_text)

In [ ]:
# def clean_text(text):
#     tokens = word_tokenize(text.lower())
#     stop_words = set(stopwords.words('english'))
#     filtered_tokens = [token for token in tokens if token not in stop_words]
#     lemmatizer = WordNetLemmatizer()
#     lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
#     table = str.maketrans('', '', string.punctuation)
#     cleaned_text = [token.translate(table) for token in lemmatized_tokens]
#     cleaned_text = ' '.join(cleaned_text)

#     return cleaned_text

# df['job'] = df['job'].apply(clean_text)

In [ ]:
# lemmatizer = WordNetLemmatizer()

# def preprocess_text(text):
#     text = re.sub('[^A-Za-z0-9]+', '', text)
#     words = nltk.word_tokenize(text)
#     words = [word for word in words if word not in stopwords.words('english')]
#     words = [lemmatizer.lemmatize(word) for word in words]
#     return ' '.join(words)

# df['job_processed'] = df['job'].apply(preprocess_text)

In [ ]:
df.loc[:,"clean_text"]

0        videograph / editor / photograph work close cr...
1        buyer charg expand vendor manag purchas networ...
2        look digit project manag lifecycl highest reve...
3        qubit: cut edg big data engineeringqubit disru...
4        adform premier partner medium agenc trade desk...
                               ...                        
12836    purposerespons sell skincar product within ran...
12837    mindwork look senior digit account manag join ...
12838    schroeder solut full-servic interior compani f...
12839    + job open websit relev job feel free search w...
12840    client lead manufactur compani medic devic loo...
Name: clean_text, Length: 12841, dtype: object

In [ ]:
dft.loc[:,"clean_text"]

0       seek qualifi candid full-tim superintend build...
1       look chang happi give u call network close ser...
2       companyw ticketscript - european market leader...
3       - must extens knowledg cm framework like wordp...
4       look program manag help support oem custom pos...
                              ...                        
3193    proficio mortgag ventur wholly-own subsidiari ...
3194    * ultra luxuri american cruis compani urgent l...
3195    look full-tim work great benefit would like ch...
3196    westview financi servic locat hampton va full-...
3197    funnelback excit grow compani headquart canber...
Name: clean_text, Length: 3198, dtype: object

In [ ]:
print(df['clean_text'][443])

clece care servic requir experienc home care assist immedi long-term work - view offer guarante hour hrs+ perman contract show commit role client excel opportun home care worker hampshir area gain secur steadi work genuin opportun develop career progress within rapidli grow group care assistant/car worker clece care help local elderli disabl client remain independ home provid person domest support - includ clean shop help feed person care companionship - also essenti care worker understand import role commun posit differ make peopl live applic requir transport due client locat ideal previou home care experi howev provid full train demonstr posit attitud desir work care sector mani avail hour flexibl avail howev also vacanc part time care assist - includ weekend evening/night visit good pay rate travel allowanceful train - includ nvq equivalentholiday payongo support teamscontinu learn developmentcar prospect rapidli grow groupincent scheme refer friends/colleaguespleas appli today disc

In [ ]:
dft.loc[:,"clean_text"]

0       seek qualifi candid full-tim superintend build...
1       look chang happi give u call network close ser...
2       companyw ticketscript - european market leader...
3       - must extens knowledg cm framework like wordp...
4       look program manag help support oem custom pos...
                              ...                        
3193    proficio mortgag ventur wholly-own subsidiari ...
3194    * ultra luxuri american cruis compani urgent l...
3195    look full-tim work great benefit would like ch...
3196    westview financi servic locat hampton va full-...
3197    funnelback excit grow compani headquart canber...
Name: clean_text, Length: 3198, dtype: object

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

label = le.fit_transform(df['fake?'])
df['fake?'] = label

In [ ]:
le = LabelEncoder()

label = le.fit_transform(dft['fake?'])
dft['fake?'] = label

In [ ]:
df.head()

,telecommuting,company logo exist?,has_questions,fake?,job,clean_text
0,0,1,1,0,The Videographer / Editor / Photographer will ...,videograph / editor / photograph work close cr...
1,0,1,1,0,Buyer will be in charge of expanding our vendo...,buyer charg expand vendor manag purchas networ...
2,0,1,1,0,We are looking for a Digital Project Manager t...,look digit project manag lifecycl highest reve...
3,0,1,1,0,"Qubit: Cutting Edge Big Data EngineeringQubit,...",qubit: cut edg big data engineeringqubit disru...
4,0,1,0,0,Adform is a premier partner for media agencies...,adform premier partner medium agenc trade desk...


In [ ]:
dft.head()

,telecommuting,company logo exist?,has_questions,fake?,job,clean_text
0,0,0,1,0,Seeking qualified candidates for a full-time s...,seek qualifi candid full-tim superintend build...
1,0,1,1,0,Looking for a change? Not happy where you are...,look chang happi give u call network close ser...
2,0,1,0,0,About the CompanyWe are ticketscript - the Eur...,companyw ticketscript - european market leader...
3,0,1,0,0,#NAME? - Must have extensive knowledge of CMS ...,- must extens knowledg cm framework like wordp...
4,0,1,1,0,We are looking for a Program Manager who can h...,look program manag help support oem custom pos...


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['clean_text'], df['fake?'], test_size=0.26, random_state=47)

In [ ]:
X=dft['clean_text']
y=dft["fake?"]

In [ ]:
vectorizer_bow = CountVectorizer()
X_train_bow = vectorizer_bow.fit_transform(X_train)
X_test_bow = vectorizer_bow.transform(X_test)
xtestbow=vectorizer_bow.transform(X)
yt=y

In [ ]:
vectorizer_tfidf = TfidfVectorizer()
X_train_tfidf = vectorizer_tfidf.fit_transform(X_train)
X_test_tfidf = vectorizer_tfidf.transform(X_test)
xtestidf=vectorizer_tfidf.transform(X)

In [ ]:
logit = LogisticRegression(max_iter = 10000)
print(logit.fit(xtestidf,yt))
print(logit.score(xtestidf,yt))

LogisticRegression(max_iter=10000)
0.9580988117573483


In [ ]:
myscaler = StandardScaler(with_mean=False).fit(xtestidf)
X_stdized = myscaler.transform(xtestidf)

In [ ]:
def create_my_model(learnRate, dropoutRate):
    # Creating the model
    mymodel = Sequential()
    mymodel.add(Dense(6, input_dim = 6, kernel_initializer = 'normal', activation = 'relu'))
    mymodel.add(Dropout(dropoutRate))
    mymodel.add(Dense(3, input_dim = 6, kernel_initializer = 'normal', activation = 'relu'))
    mymodel.add(Dropout(dropoutRate))
    mymodel.add(Dense(1, activation = 'sigmoid'))

    # Compile the model
    myadam = Adam(learning_rate = learnRate)
    mymodel.compile(loss = 'binary_crossentropy', optimizer = myadam, metrics = ['accuracy'])
    return mymodel

# Creating the model object
mymodel = KerasClassifier(build_fn = create_my_model, verbose = 1)

<ipython-input-58-a71d405712aa>:16: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  mymodel = KerasClassifier(build_fn = create_my_model, verbose = 1)


In [ ]:
# Logistic Regression
lr = LogisticRegression()
lr.fit(X_train_bow, y_train)
y_pred_lr = lr.predict(X_test_bow)
yp=lr.predict(xtestbow)
print('Logistic Regression (Bag of Words)')
print('Accuracy:', accuracy_score(y_test, y_pred_lr))
print('Precision:', precision_score(y_test, y_pred_lr))
print('Recall:', recall_score(y_test, y_pred_lr))
print('F1 Score:', f1_score(y_test, y_pred_lr))
print('Confusion Matrix:\n', confusion_matrix(y_test, y_pred_lr))



lr = LogisticRegression()
lr.fit(X_train_tfidf, y_train)
y_pred_lr = lr.predict(X_test_tfidf)
print('Logistic Regression (TF-IDF)')
print('Accuracy:', accuracy_score(y_test, y_pred_lr))
print('Precision:', precision_score(y_test, y_pred_lr))
print('Recall:', recall_score(y_test, y_pred_lr))
print('F1 Score:', f1_score(y_test, y_pred_lr))
print('Confusion Matrix:\n', confusion_matrix(y_test, y_pred_lr))


# ypb=lr.predict(xtestbow)
# print('Logistic Regression (Bag of Words Evaluation)')
# print('Accuracy:', accuracy_score(yt, ypb))
# print('Precision:', precision_score(yt, ypb))
# print('Recall:', recall_score(yt, ypb))
# print('F1 Score:', f1_score(yt, ypb))
# print('Confusion Matrix:\n', confusion_matrix(yt, ypb))


# ypb=lr.predict(xtestidf)
# print('Logistic Regression (TF-IDF Evaluation)')
# print('Accuracy:', accuracy_score(yt, ypb))
# print('Precision:', precision_score(yt, ypb))
# print('Recall:', recall_score(yt, ypb))
# print('F1 Score:', f1_score(yt, ypb))
# print('Confusion Matrix:\n', confusion_matrix(yt, ypb))


# Decision Tree
dt = DecisionTreeClassifier()
dt.fit(X_train_bow, y_train)
y_pred_dt = dt.predict(X_test_bow)
print('Decision Tree (Bag of Words)')
print('Accuracy:', accuracy_score(y_test, y_pred_dt))
print('Precision:', precision_score(y_test, y_pred_dt))
print('Recall:', recall_score(y_test, y_pred_dt))
print('F1 Score:', f1_score(y_test, y_pred_dt))
print('Confusion Matrix:\n', confusion_matrix(y_test, y_pred_dt))

dt = DecisionTreeClassifier()
dt.fit(X_train_tfidf, y_train)
y_pred_dt = dt.predict(X_test_tfidf)
print('Decision Tree (TF-IDF)')
print('Accuracy:', accuracy_score(y_test, y_pred_dt))
print('Precision:', precision_score(y_test, y_pred_dt))
print('Recall:', recall_score(y_test, y_pred_dt))
print('F1 Score:', f1_score(y_test, y_pred_dt))
print('Confusion Matrix:\n', confusion_matrix(y_test, y_pred_dt))

# dt = DecisionTreeClassifier()
# ypdt = dt.predict(xtestbow)
# print('Decision Tree (Bag of Words Evaluation)')
# print('Accuracy:', accuracy_score(yt, ypdt))
# print('Precision:', precision_score(yt, ypdt))
# print('Recall:', recall_score(yt, ypdt))
# print('F1 Score:', f1_score(yt, ypdt))
# print('Confusion Matrix:\n', confusion_matrix(yt, ypdt))

# dt = DecisionTreeClassifier()
# ypdt = dt.predict(xtestidf)
# print('Decision Tree (Bag of Words Evaluation)')
# print('Accuracy:', accuracy_score(yt, ypdt))
# print('Precision:', precision_score(yt, ypdt))
# print('Recall:', recall_score(yt, ypdt))
# print('F1 Score:', f1_score(yt, ypdt))
# print('Confusion Matrix:\n', confusion_matrix(yt, ypdt))


# Random Forest
rf = RandomForestClassifier()
rf.fit(X_train_bow, y_train)
y_pred_rf = rf.predict(X_test_bow)
print('Random Forest (Bag of Words)')
print('Accuracy:', accuracy_score(y_test, y_pred_rf))
print('Precision:', precision_score(y_test, y_pred_rf))
print('Recall:', recall_score(y_test, y_pred_rf))
print('F1 Score:', f1_score(y_test, y_pred_rf))
print('Confusion Matrix:\n', confusion_matrix(y_test, y_pred_rf))


#test
rf = RandomForestClassifier()
rf.fit(X_train_tfidf, y_train)
y_pred_rf = rf.predict(X_test_tfidf)
print('Random Forest (testbow)')
print('Accuracy:', accuracy_score(y_test, y_pred_rf))
print('Precision:', precision_score(y_test, y_pred_rf))
print('Recall:', recall_score(y_test, y_pred_rf))
print('F1 Score:', f1_score(y_test, y_pred_rf))
print('Confusion Matrix:\n', confusion_matrix(y_test, y_pred_rf))
# rf = RandomForestClassifier()
# yprf = rf.predict(xtestbow)
# print('Random Forest (Bag of Words Evaluation)')
# print('Accuracy:', accuracy_score(yt, yprf))
# print('Precision:', precision_score(yt, yprf))
# print('Recall:', recall_score(yt, yprf))
# print('F1 Score:', f1_score(yt, yprf))
# print('Confusion Matrix:\n', confusion_matrix(yt, yprf))
print('Confusion Matrix:\n', confusion_matrix(y_test, y_pred_rf))
# rf = RandomForestClassifier()
# yprf = rf.predict(xtestidf)
# print('Random Forest (TF-IDF Evaluation)')
# print('Accuracy:', accuracy_score(yt, yprf))
# print('Precision:', precision_score(yt, yprf))
# print('Recall:', recall_score(yt, yprf))
# print('F1 Score:', f1_score(yt, yprf))
# print('Confusion Matrix:\n', confusion_matrix(yt, yprf))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression (Bag of Words)
Accuracy: 0.9745432764300689
Precision: 0.7414965986394558
Recall: 0.6987179487179487
F1 Score: 0.7194719471947195
Confusion Matrix:
 [[3145   38]
 [  47  109]]
Logistic Regression (TF-IDF)
Accuracy: 0.9667565139263252
Precision: 0.9591836734693877
Recall: 0.30128205128205127
F1 Score: 0.4585365853658537
Confusion Matrix:
 [[3181    2]
 [ 109   47]]
Decision Tree (Bag of Words)
Accuracy: 0.9742437855645403
Precision: 0.7573529411764706
Recall: 0.6602564102564102
F1 Score: 0.7054794520547946
Confusion Matrix:
 [[3150   33]
 [  53  103]]
Decision Tree (TF-IDF)
Accuracy: 0.9691524408505541
Precision: 0.6625766871165644
Recall: 0.6923076923076923
F1 Score: 0.677115987460815
Confusion Matrix:
 [[3128   55]
 [  48  108]]
Random Forest (Bag of Words)
Accuracy: 0.9787361485474693
Precision: 1.0
Recall: 0.5448717948717948
F1 Score: 0.7053941908713693
Confusion Matrix:
 [[3183    0]
 [  71   85]]
Random Forest (TF-IDF)
Accuracy: 0.9778376759508834
Precision: 1.